# Continuation of QC - milestone num. 5 - issue num. 2
CellBender recognised a lot of empty cells, since we've chosen to not use CellBender, and the most reasonable thing is that most of the TP detection of empty cells might be cells we classified stromal.

For this reason we will investigate stromal cells:
1. Comparing number of reads in stromal cells and immune/tumor cells.
2. After analysing the finding of the previous step we will decide whether to check for prominent overlap between stromal cell and cells classified empty by CellBender.   


In [3]:
import sys
lib = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities\droplet_dataset'
lib2 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities'
lib3 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\data_analysis'
lib4 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy'
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
import os
from os.path import join
import sklearn
from utilities.droplet_dataset import *
# from utilities import *
from matplotlib import pyplot
import numpy as np
import scipy
import pickle
import matplotlib.pyplot as plt
import pickle
from statsmodels.stats.proportion import proportions_ztest
import pandas as pd
import random
from scipy.stats import pearsonr
from matplotlib.pyplot import figure
import time
import matplotlib.pyplot as plt
import copy
from scipy import stats
from sklearn.cluster import KMeans
from collections import Counter
from PIL import Image, ImageColor
from termcolor import colored
from sklearn.manifold import TSNE
from DL.Mars_seq_DL.data_loading import extract_droplet_data_from_pickle

In [4]:
def extract_sample(sample_id):
    """
    Extracts one of the samples from PC
    :param sample_id: id of rna sample (Mi)
    :return: rna_sample
    """
    data_path = join(SAMPLES, f'{sample_id}')
    rna_sample = extract_droplet_data_from_pickle(data_path)
    print(colored(f'sample id {sample_id}', 'blue'))
#     print(f'count shape {rna_sample.counts.shape}')
    print(f'number of cells {rna_sample.number_of_cells}', end='\t')
    print(f'number of genes {rna_sample.number_of_genes}')
    return rna_sample

# 1. Comparing number of reads in stromal cells and immune/tumor cells:

In [8]:
SAMPLES = r'D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\21.2.21'

df = pd.DataFrame(columns=["sample", "immune&stromal_p_val", "immune_test", "tumor&stromal_p_val", "tumor_test",
                      'immune_mean', 'tumor_mean', 'stromal_mean',
                      'immune_median', 'tumor_median', 'stromal_median'])

samples = [subfolder for subfolder in os.listdir(SAMPLES)]
# create_folder(OUTPUT_PATH)
for sample in [s for s in samples if not 'csv' in s]:
    
    # Extracts one of the samples from PC
    rna_sample = extract_sample(sample)
    _rna_sample = RNAseq_Sample(counts=rna_sample.counts, gene_names=rna_sample.gene_names, barcodes=rna_sample.barcodes, 
                            features=rna_sample.features, cells_information=rna_sample.cells_information)
    rna_sample = _rna_sample  
    
    ###   Remove garbage cells  ###
    rna_sample = rna_sample[[not aa for aa in rna_sample.cells_information.getattr('should_be_removed')]]

    ###  Normalize cells  ###
    rna_sample.normalize_data()
    
    ###   Filter cancer/immune/stromal cells  ###
    ss = rna_sample[rna_sample.cells_information.getattr('is_stromal')].counts.sum(axis=1)
    ns = rna_sample[[not aa for aa in rna_sample.cells_information.getattr('is_stromal')]].counts.sum(axis=1)
    ii = rna_sample[rna_sample.cells_information.getattr('is_immune')].counts.sum(axis=1)
    tt = rna_sample[rna_sample.cells_information.getattr('is_cancer')].counts.sum(axis=1)
    re = rna_sample[rna_sample.cells_information.getattr('should_be_removed')].counts.sum(axis=1)

    
    ###  Test distributions ###
    if stats.shapiro(ss)[1] < 0.05 or stats.shapiro(ii)[1] < 0.05:
        immune_stromal_p_val = scipy.stats.mannwhitneyu(ss, ii, alternative='less')[1]
        imm_test = "mannwhitneyu"
    else:
        immune_stromal_p_val = stats.ttest_ind(ss, ii, alternative='less')[1]
        imm_test = "ttest"

    if stats.shapiro(ss)[1] < 0.05 or stats.shapiro(tt)[1] < 0.05:
        tumor_stromal_p_val = scipy.stats.mannwhitneyu(ss, tt, alternative='less')[1]
        tu_test = "mannwhitneyu"
    else:
        tumor_stromal_p_val = stats.ttest_ind(ss, tt, alternative='less')[1]
        tu_test = "ttest"

    ###  Extract distribution attributes ###
    immune_mean = ii.mean()
    tumor_mean = tt.mean()
    stromal_mean = ss.mean()

    immune_median = np.median(ii)
    tumor_median = np.median(tt)
    stromal_median = np.median(ss)

    ###  Update DataFrame ###
    df = df.append(pd.DataFrame([[sample.replace('.pkl',''), 
                             immune_stromal_p_val, 
                             imm_test, 
                             tumor_stromal_p_val, 
                             tu_test,
                             immune_mean,
                            tumor_mean,
                            stromal_mean,
                            immune_median,
                            tumor_median,
                            stromal_median]],columns=df.columns))
df

sample id M100.pkl
number of cells 4055	number of genes 20765


D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities\droplet_dataset.py:200: RuntimeWarning: divide by zero encountered in log2
  normalized_cells = np.log2((self.counts / scaling_factors[:, None])) + 1


sample id M101.pkl
number of cells 5141	number of genes 22423
sample id M102.pkl
number of cells 22694	number of genes 21324
sample id M103.pkl
number of cells 2453	number of genes 20400
sample id M104.pkl
number of cells 4944	number of genes 23198
sample id M105.pkl
number of cells 14312	number of genes 22537
sample id M106.pkl
number of cells 5136	number of genes 22752
sample id M107.pkl
number of cells 8035	number of genes 22683
sample id M108.pkl
number of cells 23821	number of genes 20906
sample id M109.pkl
number of cells 7832	number of genes 21987
sample id M110.pkl
number of cells 4307	number of genes 22327
sample id M111.pkl
number of cells 6582	number of genes 22219
sample id M112.pkl
number of cells 4876	number of genes 21540
sample id M114.pkl
number of cells 10407	number of genes 24214
sample id M115.pkl
number of cells 6047	number of genes 21681
sample id M116.pkl
number of cells 3601	number of genes 19183
sample id M118.pkl
number of cells 1687	number of genes 20194
samp

,sample,immune&stromal_p_val,immune_test,tumor&stromal_p_val,tumor_test,immune_mean,tumor_mean,stromal_mean,immune_median,tumor_median,stromal_median
0,M100,1.826444e-30,mannwhitneyu,9.284181e-77,mannwhitneyu,6614.849621,12547.576794,3675.345569,6309.645622,13438.004715,3337.252086
0,M101,1.000000e+00,mannwhitneyu,6.644373e-40,mannwhitneyu,9363.771762,13065.818664,10357.538138,8022.678602,13549.635433,10904.713107
0,M102,5.058933e-140,mannwhitneyu,2.436438e-251,mannwhitneyu,6272.242232,5836.506321,4203.595085,6168.877030,5059.312325,4030.161321
0,M103,2.758837e-01,mannwhitneyu,1.232939e-03,mannwhitneyu,9350.936789,10414.521278,8884.103472,8433.499283,10445.296604,10028.754643
0,M104,2.664075e-29,mannwhitneyu,0.000000e+00,mannwhitneyu,6862.886901,14994.454714,4699.116424,6274.509301,16078.384947,3866.590233
0,M105,2.324864e-19,mannwhitneyu,1.217774e-69,mannwhitneyu,4745.867302,7254.924993,4459.126734,4530.225602,6818.472468,3679.575272
0,M106,1.000000e+00,mannwhitneyu,1.030151e-06,mannwhitneyu,7618.163549,12888.101792,11389.341084,7193.040742,13847.387355,11631.013660
0,M107,1.000000e+00,mannwhitneyu,3.848636e-23,mannwhitneyu,6900.230265,10601.022720,8711.196273,6670.057123,10852.221651,9346.972849
0,M108,8.441675e-14,mannwhitneyu,7.570809e-201,mannwhitneyu,3076.731041,3028.651356,2393.388737,2779.035325,2573.274404,2247.964563
0,M109,1.622292e-01,mannwhitneyu,2.742991e-79,mannwhitneyu,5349.498513,11230.636801,5565.413650,4777.806743,12501.339677,4684.943475


In [9]:
df.to_excel(r'D:\Technion studies\Keren Laboratory\python_playground\outputs\temporal garbage\QC_stromal_reads_distribution.xlsx', index='False')

# 2. Checking for prominent overlap between stromal cell and cells classified empty by CellBender.

Decided not to do. not completed.

In [211]:
cellbender_rna_sample_path = r'D:\Technion studies\Keren Laboratory\python_playground\outputs\CellBender\update_runs\empty_kept'
cnv_rna_sample_path = r'D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\21.2.21'

samples = [subfolder for subfolder in os.listdir(cellbender_rna_sample_path)]
proportion_test_df = pd.DataFrame(columns=['sample', 'n_not_stromal_cells', 'n_empty_not_stromal_cells', 'ratio_not_stromal',
                                          'n_stromal_cells', 'n_empty_stromal_cells', 'ratio_stromal', 'proportions_ztest_pval'])


# create_folder(OUTPUT_PATH)
for sample in [s for s in samples if not 'csv' in s]:    
    cellbender_rna_sample = extract_sample(join(cellbender_rna_sample_path,sample,f'{sample}.pkl'))
    empty_cell_barcodes = cellbender_rna_sample[[a.is_CelBender_empty for a in cellbender_rna_sample.cells_information]].barcodes

    cnv_rna_sample = extract_sample(join(cnv_rna_sample_path,f'{sample}.pkl'))
    ###   Remove garbage cells  ###
    cnv_rna_sample = cnv_rna_sample[[not aa for aa in cnv_rna_sample.cells_information.getattr('should_be_removed')]]
    
    
    ###   filter stromal cells  ###
    is_stromal_indices = cnv_rna_sample.cells_information.getattr('is_stromal')

    stromal_cells = cnv_rna_sample[is_stromal_indices]
    not_stromal_cells = cnv_rna_sample[[not aa for aa in is_stromal_indices]]

    ###   take empty cells  ###
    empty_not_stromal = not_stromal_cells.get_subset_by_barcodes(intersection_of_lists(empty_cell_barcodes, not_stromal_cells.barcodes))
    empty_stromal = stromal_cells.get_subset_by_barcodes(intersection_of_lists(empty_cell_barcodes, stromal_cells.barcodes))


    print(empty_not_stromal.number_of_cells)
    print(not_stromal_cells.number_of_cells)
    print(empty_not_stromal.number_of_cells / not_stromal_cells.number_of_cells)

    print()
    print(empty_stromal.number_of_cells)
    print(stromal_cells.number_of_cells)
    print(empty_stromal.number_of_cells / stromal_cells.number_of_cells)
    
    count = np.array([empty_not_stromal.number_of_cells, empty_stromal.number_of_cells]) #12
    nobs = np.array([not_stromal_cells.number_of_cells, stromal_cells.number_of_cells]) #180
    p_val = proportions_ztest(count, nobs, alternative='smaller')[1]
    print(p_val)
    
    proportion_test_df = proportion_test_df.append(pd.DataFrame([[sample, not_stromal_cells.number_of_cells,
                                                                empty_not_stromal.number_of_cells,
                                                                empty_not_stromal.number_of_cells/ not_stromal_cells.number_of_cells,
                                                                stromal_cells.number_of_cells,
                                                                empty_stromal.number_of_cells,
                                                                (empty_stromal.number_of_cells/ stromal_cells.number_of_cells),
                                                                p_val]],columns=proportion_test_df.columns))
proportion_test_df

sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\CellBender\update_runs\empty_kept\M100\M100.pkl
number of cells 4055	number of genes 20765
sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\21.2.21\M100.pkl
number of cells 4055	number of genes 20765
847
3162
0.2678684376976597

150
153
0.9803921568627451
6.648745089589019e-79
sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\CellBender\update_runs\empty_kept\M101\M101.pkl
number of cells 5141	number of genes 22423
sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\21.2.21\M101.pkl
number of cells 5141	number of genes 22423
723
1796
0.40256124721603564

832
2590
0.32123552123552124
0.9999999845654544
sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\CellBender\update_runs\empty_kept\M102\M102.pkl
number of cells 22694	number of genes 21324
sample id D:\Technion studies\Keren Laborat

sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\CellBender\update_runs\empty_kept\M123\M123.pkl
number of cells 6743	number of genes 23673
sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\21.2.21\M123.pkl
number of cells 6743	number of genes 23673
0
2811
0.0

0
1732
0.0
nan


C:\Users\itay\AppData\Roaming\Python\Python37\site-packages\statsmodels\stats\weightstats.py:788: RuntimeWarning: invalid value encountered in double_scalars
  zstat = value / std
c:\users\itay\appdata\local\programs\python\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\CellBender\update_runs\empty_kept\M124\M124.pkl
number of cells 5148	number of genes 20889
sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\21.2.21\M124.pkl
number of cells 5148	number of genes 20889
3779
4419
0.8551708531341933

109
140
0.7785714285714286
0.994109144229045
sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\CellBender\update_runs\empty_kept\M125\M125.pkl
number of cells 507	number of genes 14842
sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\21.2.21\M125.pkl
number of cells 507	number of genes 14842
0
135
0.0

3
202
0.01485148514851485
0.07746848048568647
sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\CellBender\update_runs\empty_kept\M126\M126.pkl
number of cells 3125	number of genes 19571
sample id D:\Technion studies\Keren Laboratory\python_playground\outpu

sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\21.2.21\M145.pkl
number of cells 8911	number of genes 32738
203
4670
0.04346895074946467

279
3403
0.08198648251542756
2.7445143699291913e-13
sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\CellBender\update_runs\empty_kept\M146\M146.pkl
number of cells 8281	number of genes 32738
sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\21.2.21\M146.pkl
number of cells 8281	number of genes 32738
682
6520
0.10460122699386504

112
167
0.6706586826347305
9.544512730455601e-111
sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\CellBender\update_runs\empty_kept\M97\M97.pkl
number of cells 4296	number of genes 21462
sample id D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\21.2.21\M97.pkl
number of cells 4296	number of genes 21462
1331
3212
0.4143835616438356

131
255
0.51372549019

,sample,n_not_stromal_cells,n_empty_not_stromal_cells,ratio_not_stromal,n_stromal_cells,n_empty_stromal_cells,ratio_stromal,proportions_ztest_pval
0,M100,3162,847,0.267868,153,150,0.980392,6.648745e-79
0,M101,1796,723,0.402561,2590,832,0.321236,1.000000e+00
0,M102,18852,4,0.000212,1420,0,0.000000,7.084825e-01
0,M103,518,169,0.326255,1561,477,0.305573,8.109288e-01
0,M104,2520,315,0.125000,1886,1533,0.812831,0.000000e+00
0,M105,9683,1629,0.168233,723,183,0.253112,3.208679e-09
0,M106,4538,1870,0.412076,168,128,0.761905,1.045646e-19
0,M107,6656,2201,0.330679,497,244,0.490946,1.847116e-13
0,M108,13625,2,0.000147,4026,1,0.000248,3.319734e-01
0,M109,5619,822,0.146289,375,40,0.106667,9.828763e-01


In [212]:
save_path = r'D:\Technion studies\Keren Laboratory\python_playground\outputs\temporal garbage\QC_CellBender_stromal_empty.xlsx'
proportion_test_df.reset_index(drop=True).to_excel(save_path, index='False')

In [2]:
scipy.__version__

'1.6.1'

# DEBUG
Checking scaling factor for normalization, 10K or 100K or 1M. for what values we will get number under zerows after normalization be done.

In [10]:
for sample in [s for s in samples if not 'csv' in s]:
    # Extracts one of the samples from PC
    rna_sample = extract_sample(sample)    
    #10K ----------------------------------------------------------
    sum_cells = rna_sample.counts.sum(axis=1)  # for each cell
    scaling_factors = sum_cells / 10000

    normalized_cells = np.log2((rna_sample.counts / scaling_factors[:, None])) + 1
    normalized_cells[np.isneginf(normalized_cells)] = 0
    checked = normalized_cells.flatten()
    # plt.boxplot(checked.flatten())
    print("10K")
    print(checked[checked<0].shape)
    print("max", checked.max())
    print("min", checked.min())
    print("mean", checked.mean())
    print("median", np.median(checked))
    
    #100K ----------------------------------------------------------
    sum_cells = rna_sample.counts.sum(axis=1)  # for each cell
    scaling_factors = sum_cells / 100000 

    normalized_cells = np.log2((rna_sample.counts / scaling_factors[:, None])) + 1
    normalized_cells[np.isneginf(normalized_cells)] = 0
    checked = normalized_cells.flatten()
    # plt.boxplot(checked.flatten())
    print("100K")
    print(checked[checked<0].shape)
    print("max", checked.max())
    print("min", checked.min())
    print("mean", checked.mean())
    print("median", np.median(checked))
    
    
    
    #1M ----------------------------------------------------------
    sum_cells = rna_sample.counts.sum(axis=1)  # for each cell
    scaling_factors = sum_cells / 1000000 # 1M

    normalized_cells = np.log2((rna_sample.counts / scaling_factors[:, None])) + 1
    normalized_cells[np.isneginf(normalized_cells)] = 0
    checked = normalized_cells.flatten()
    # plt.boxplot(checked.flatten())
    print("1m")
    print(checked[checked<0].shape)
    print("max", checked.max())
    print("min", checked.min())
    print("mean", checked.mean())
    print("median", np.median(checked))

sample id M100.pkl
number of cells 4055	number of genes 20765


c:\users\itay\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log2
  


10K
(1218162,)
max 14.226073313757064
min -1.4085216627056063
mean 0.15403869421693417
median 0.0


c:\users\itay\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log2


100K
(0,)
max 17.548001408644424
min 0.0
mean 0.5074582786633369
median 0.0


c:\users\itay\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in log2


1m
(0,)
max 20.86992950353179
min 0.0
mean 0.8608778631097372
median 0.0
sample id M101.pkl
number of cells 5141	number of genes 22423
10K
(352577,)
max 14.166033822961197
min -1.1143337059750849
mean 0.18149726659781787
median 0.0
100K
(0,)
max 17.48796191784856
min 0.0
mean 0.45647873670831496
median 0.0
1m
(0,)
max 20.80989001273592
min 0.0
mean 0.7314602068188105
median 0.0
sample id M102.pkl
number of cells 22694	number of genes 21324
10K
(204306,)
max 14.081022566168777
min -1.1729354878909697
mean 0.13991125385407135
median 0.0
100K
(0,)
max 17.40295066105614
min 0.0
mean 0.266126374895234
median 0.0
1m
(0,)
max 20.7248787559435
min 0.0
mean 0.39234149593639694
median 0.0
sample id M103.pkl
number of cells 2453	number of genes 20400
10K
(54017,)
max 14.178358241622208
min -1.0519288849942354
mean 0.17071930758539533
median 0.0
100K
(0,)
max 17.50028633650957
min 0.0
mean 0.3984589032174155
median 0.0
1m
(0,)
max 20.822214431396933
min 0.0
mean 0.6261984988494389
median 0.0
sampl

mean 0.5568772206441458
median 0.0
sample id M130.pkl
number of cells 5981	number of genes 23986
10K
(2831317,)
max 12.489723300887876
min -2.8885679234049433
mean 0.1472241790660988
median 0.0
100K
(0,)
max 15.81165139577524
min 0.0
mean 0.5041100203379375
median 0.0
1m
(0,)
max 19.1335794906626
min 0.0
mean 0.8609958616097731
median 0.0
sample id M131.pkl
number of cells 4722	number of genes 22341
10K
(1030427,)
max 13.461177007226857
min -2.6760191133486337
mean 0.1789144333973275
median 0.0
100K
(0,)
max 16.78310510211422
min 0.0
mean 0.4985031172197988
median 0.0
1m
(0,)
max 20.105033197001582
min 0.0
mean 0.8180918010422725
median 0.0
sample id M132.pkl
number of cells 5949	number of genes 22320
10K
(2277664,)
max 13.526380340598728
min -1.5716768099709313
mean 0.14091954050406133
median 0.0
100K
(0,)
max 16.84830843548609
min 0.0
mean 0.4486738461543644
median 0.0
1m
(0,)
max 20.17023653037345
min 0.0
mean 0.7564281518046637
median 0.0
sample id M133.pkl
number of cells 5696	num

In [199]:
### proportion test 

from statsmodels.stats.proportion import proportions_ztest
count = sum(is_stromal) #12
nobs = len(is_stromal) #180
value = sum(cnv_rna_sample.cells_information.getattr('is_stromal'))/cnv_rna_sample.number_of_cells
proportions_ztest(count, nobs, value, alternative='two-sided')


C:\Users\itay\AppData\Roaming\Python\Python37\site-packages\statsmodels\stats\proportion.py:840: RuntimeWarning: invalid value encountered in double_scalars
  prop = count * 1. / nobs
C:\Users\itay\AppData\Roaming\Python\Python37\site-packages\statsmodels\stats\proportion.py:854: RuntimeWarning: invalid value encountered in double_scalars
  p_pooled = np.sum(count) * 1. / np.sum(nobs)
C:\Users\itay\AppData\Roaming\Python\Python37\site-packages\statsmodels\stats\proportion.py:856: RuntimeWarning: divide by zero encountered in true_divide
  nobs_fact = np.sum(1. / nobs)
c:\users\itay\appdata\local\programs\python\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


(nan, nan)

In [23]:
l = [(4, 10), (6, 18), (6, 18), (3, 13)]

# sorted(l, key=lambda x: x[1])
list(set(l))

[(4, 10), (6, 18), (3, 13)]

In [28]:
l2 = [1,2,3]
list(zip(l2, l2))

[(1, 1), (2, 2), (3, 3)]